<a href="https://colab.research.google.com/github/johnmelwin/ResearchProject1/blob/main/5_BERT_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install accelerate
!pip install transformers[torch] -U


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 7.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
import json, re
import torch
from transformers import BertTokenizer
from torch.utils.data import Dataset, DataLoader

# Sample data path
data_path = "/content/GPT4_training_dataset_binary_JHOT_1000.jsonl"

# Load and preprocess data
def load_data(data_path, max_examples=1000):
    texts = []
    labels = []

    with open(data_path, 'r') as file:
        for line in file:
            if len(texts) >= max_examples:  # Stop after loading max_examples
                break
            entry = json.loads(line)
            user_msg = entry['messages'][1]['content']
            assistant_msg = entry['messages'][2]['content']

            # Regular expression to extract the relevant parts
            regex_pattern = r"requirement: '([^']+)' is linked with method name: '([^']+)' and method implementation: '([^']+)'"
            matches = re.search(regex_pattern, user_msg, re.IGNORECASE)

            if matches:
                requirement, method_name, method_implementation = matches.groups()
                # Reformat the extracted information
                formatted_text = f"Requirement: '{requirement}'. Method: '{method_name}'. Implementation: '{method_implementation}'."
                texts.append(formatted_text)
            else:
                # If the pattern does not match, append the original message (or handle as needed)
                texts.append(user_msg)

            # Extracting label
            label = 1 if assistant_msg.strip() == "True" else 0
            labels.append(label)

    return texts, labels

texts, labels = load_data(data_path, 1000)

# Tokenize the texts
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
encodings = tokenizer(texts, truncation=True, padding=True, max_length=512, return_tensors="pt")

# Create a custom dataset
class TraceabilityDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

dataset = TraceabilityDataset(encodings, labels)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

# Load BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,  # Reduced from 3 to 2
    per_device_train_batch_size=8,  # Increased from 8 to 16, adjust based on your GPU memory
    warmup_steps=300,  # Reduced to adjust to the reduced number of epochs
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    # Adding more aggressive learning rate scheduling can help
    learning_rate=2e-5,
)


# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

# Train the model
trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
10,0.845300
20,0.732300
30,0.548100
40,0.465200
50,0.482200
60,0.380700
70,0.363700
80,0.452300
90,0.432700
100,0.515400


TrainOutput(global_step=375, training_loss=0.4408934523264567, metrics={'train_runtime': 156.2979, 'train_samples_per_second': 19.194, 'train_steps_per_second': 2.399, 'total_flos': 789333166080000.0, 'train_loss': 0.4408934523264567, 'epoch': 3.0})

In [ ]:
model.save_pretrained('./results')
tokenizer.save_pretrained('./results')


('./results/tokenizer_config.json',
 './results/special_tokens_map.json',
 './results/vocab.txt',
 './results/added_tokens.json')

In [ ]:
import json, re
import torch
from transformers import BertTokenizer

# Assuming the use of BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def load_test_data(data_path, max_examples=1000):
    texts = []

    with open(data_path, 'r') as file:
        for line in file:
            if len(texts) >= max_examples:
                break
            entry = json.loads(line)
            user_msg = entry['messages'][1]['content']

            regex_pattern = r"REQUIREMENT: '([^']+)' is linked with METHOD NAME: '([^']+)' and METHOD IMPLEMENTATION: '([^']+)'"
            matches = re.search(regex_pattern, user_msg, re.IGNORECASE)

            if matches:
                requirement, method_name, method_implementation = matches.groups()
                formatted_text = f"REQUIREMENT: '{requirement}'. METHOD: '{method_name}'. IMPLEMENTATION: '{method_implementation}'."
                texts.append(formatted_text)
            else:
                texts.append(user_msg)  # Or handle unmatched cases as needed

    return texts

data_path = "/content/GPT4_testing_dataset_binary_JHOT_1000.jsonl"  # Update this path
test_texts = load_test_data(data_path)

# Tokenize the test texts
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=512, return_tensors="pt")




In [ ]:
from transformers import BertForSequenceClassification
import torch

# Load your trained model
model = BertForSequenceClassification.from_pretrained('./results', num_labels=2)
model.eval()  # Set model to evaluation mode

# Prepare test data
input_ids = test_encodings['input_ids']
attention_mask = test_encodings['attention_mask']

# Perform inference
predictions = []
confidences = []

with torch.no_grad():
    for i in range(len(input_ids)):
        outputs = model(input_ids[i].unsqueeze(0), attention_mask=attention_mask[i].unsqueeze(0))
        logits = outputs.logits
        probs = torch.softmax(logits, dim=1)
        prediction = torch.argmax(probs, dim=1).item()
        confidence = probs[0][prediction].item()  # Extract the probability for the predicted class
        predictions.append((prediction, confidence))

# Print predictions with their confidence scores
for prediction, confidence in predictions:
    print(f"Prediction: {prediction}, Confidence: {confidence:.4f}")


Prediction: 0, Confidence: 0.9567
Prediction: 0, Confidence: 0.9312
Prediction: 0, Confidence: 0.9599
Prediction: 0, Confidence: 0.9552
Prediction: 0, Confidence: 0.9601
Prediction: 0, Confidence: 0.9582
Prediction: 0, Confidence: 0.9593
Prediction: 0, Confidence: 0.9222
Prediction: 0, Confidence: 0.9651
Prediction: 0, Confidence: 0.9584
Prediction: 1, Confidence: 0.7401
Prediction: 0, Confidence: 0.9588
Prediction: 0, Confidence: 0.8131
Prediction: 0, Confidence: 0.9543
Prediction: 0, Confidence: 0.9623
Prediction: 0, Confidence: 0.9607
Prediction: 0, Confidence: 0.8141
Prediction: 0, Confidence: 0.5027
Prediction: 0, Confidence: 0.9616
Prediction: 0, Confidence: 0.9615
Prediction: 0, Confidence: 0.9593
Prediction: 0, Confidence: 0.9601
Prediction: 0, Confidence: 0.9595
Prediction: 0, Confidence: 0.8320
Prediction: 0, Confidence: 0.9616
Prediction: 0, Confidence: 0.9520
Prediction: 0, Confidence: 0.9603
Prediction: 1, Confidence: 0.6765
Prediction: 0, Confidence: 0.8125
Prediction: 0,

In [ ]:
print (predictions)

[(0, 0.9686812162399292), (0, 0.9637270569801331), (0, 0.964241087436676), (0, 0.9615248441696167), (0, 0.9690844416618347), (0, 0.9590811133384705), (0, 0.9643620252609253), (0, 0.9599544405937195), (0, 0.959909975528717), (0, 0.9641060829162598), (0, 0.9630765914916992), (1, 0.889105498790741), (1, 0.8977599740028381), (0, 0.9632392525672913), (0, 0.9643068909645081), (0, 0.9625755548477173), (0, 0.9662074446678162), (0, 0.9577335119247437), (0, 0.9621058702468872), (0, 0.9646726250648499), (0, 0.9653958082199097), (0, 0.9516232013702393), (1, 0.9001322388648987), (0, 0.9674004316329956), (0, 0.9689872860908508), (1, 0.8954523801803589), (0, 0.9616703391075134), (0, 0.9548293352127075), (0, 0.9554323554039001), (0, 0.9667141437530518), (0, 0.965108335018158), (0, 0.9570561051368713), (0, 0.9678824543952942), (0, 0.9675856828689575), (0, 0.9496966600418091), (0, 0.958893358707428), (1, 0.8980675339698792), (0, 0.9668922424316406), (0, 0.9390162825584412), (0, 0.9647977948188782), (0, 

In [ ]:
import csv

# Assuming `predictions` is a list of tuples containing (predicted_class, confidence_score)
# Convert numerical predictions to boolean and keep confidence
boolean_predictions_with_confidence = [
    ("TRUE", confidence) if pred_class == 1 else ("FALSE", confidence)
    for pred_class, confidence in predictions
]

# Specify the CSV file name
csv_file_name = 'BERT_with_confidence_JHOT.csv'

# Write the boolean predictions and their confidence scores to the CSV file
with open(csv_file_name, mode='w', newline='') as file:
    writer = csv.writer(file)
    # Optional: Write a header row
    writer.writerow(['Prediction', 'Confidence'])
    # Write the boolean predictions with confidence
    for prediction, confidence in boolean_predictions_with_confidence:
        writer.writerow([prediction, f"{confidence:.4f}"])  # Format confidence to 4 decimal places

